In [1]:
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from aeon.datasets import  load_from_tsfile
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from aeon.transformations.collection.convolution_based import MiniRocketMultivariate
from aeon.datasets import load_basic_motions, load_gunpoint
import numpy as np

In [2]:
from tsCaptum.explainers import Feature_Ablation, Feature_Permutation, LIME, Kernel_Shap


# load regression and classification datasets

In [3]:
X_train_reg, y_train_reg = load_from_tsfile("./tsCaptum/data/AppliancesEnergy_TRAIN.ts")
X_test_reg, y_test_reg = load_from_tsfile("./tsCaptum/data/AppliancesEnergy_TEST.ts")


X_train_clf, y_train_clf = load_gunpoint(split="train")
X_test_clf, y_test_clf = load_gunpoint(split="test")


# train quant and explain it point and segmentation-wise

In [6]:

from aeon.classification.interval_based import QUANTClassifier
clf = QUANTClassifier()
clf.fit(X_train_clf, y_train_clf)
print ("QUAN accuracy is",clf.score(X_test_clf,y_test_clf), X_train_clf.shape, X_test_clf.shape,)



QUAN accuracy is 0.9933333333333333 (50, 1, 150) (150, 1, 150)


In [7]:

n_to_explain = 10
myFP4clf = Feature_Permutation(clf)
exp = myFP4clf.explain(  X_train_clf[:n_to_explain],   labels=y_test_clf[:n_to_explain], batch_size=2, n_segments=7)
print( type(exp),"unique values", np.unique(exp).shape , "\n\n")


/home/davide/workspace/PhD/captum4aeon/tsCaptum/explainers.py:92: UserWarning:  batch_size set to 2 as Feature Permutation require more than 1 sample to work
  warnings.warn(" batch_size set to 2 as Feature Permutation require more than 1 sample to work")
100%|██████████| 10/10 [00:01<00:00,  6.71it/s]

<class 'numpy.ndarray'> unique values (45,) 




In [8]:
exp = myFP4clf.explain(X_train_clf[:n_to_explain],labels=y_test_clf[:n_to_explain], batch_size=6)
print( type(exp), exp.shape, "unique values", np.unique(exp).shape , "\n\n")


/home/davide/workspace/PhD/captum4aeon/tsCaptum/explainers.py:92: UserWarning:  batch_size set to 2 as Feature Permutation require more than 1 sample to work
  warnings.warn(" batch_size set to 2 as Feature Permutation require more than 1 sample to work")
100%|██████████| 10/10 [00:24<00:00,  2.42s/it]

<class 'numpy.ndarray'> (10, 1, 150) unique values (56,) 




# kernel shap
testing normalisation 

In [9]:
import torch
n_to_explain = 20

myKernel4clf = Kernel_Shap(clf=clf, clf_type="classifier")
exp = myKernel4clf.explain( torch.tensor(X_test_clf[:n_to_explain]), labels=y_test_clf[:n_to_explain], normalise=True)
print( type(exp),"unique values", np.unique(exp).shape ,
       "\n max and min values are ", exp.max(), exp.min() )


100%|██████████| 20/20 [00:14<00:00,  1.34it/s]

<class 'numpy.ndarray'> unique values (2359,) 
 max and min values are  1.0 -1.0


In [10]:
# now without normalisation 
exp = myKernel4clf.explain( torch.tensor(X_test_clf[:n_to_explain]), labels=y_test_clf[:n_to_explain], normalise=False)
print( type(exp),"unique values", np.unique(exp).shape ,
       "\n max and min values are ", exp.max(), exp.min() )

100%|██████████| 20/20 [00:14<00:00,  1.35it/s]

<class 'numpy.ndarray'> unique values (2547,) 
 max and min values are  0.14000005 -0.08000023


In [12]:
exp = myKernel4clf.explain( torch.tensor(X_test_clf[:n_to_explain]), labels=y_test_clf[:n_to_explain], n_segments=5)
print( type(exp),"unique values", np.unique(exp).shape , "\n\n")

100%|██████████| 20/20 [00:14<00:00,  1.36it/s]

<class 'numpy.ndarray'> unique values (100,) 




# other dataset?

In [13]:
X_train_clf, y_train_clf = load_basic_motions(split="train")
X_test_clf, y_test_clf = load_basic_motions(split="test")
clf = make_pipeline(MiniRocketMultivariate(n_jobs=1),
                    StandardScaler(),LogisticRegressionCV(n_jobs=-1, max_iter=1000))

clf.fit(X_train_clf, y_train_clf)
print("accuracy is", clf.score(X_test_clf,y_test_clf) )

accuracy is 1.0


In [14]:
n_to_explain = 40
myFA4clf = Feature_Ablation(clf)
exp = myFA4clf.explain(X_train_clf[:n_to_explain],labels=y_test_clf[:n_to_explain],n_segments=7)
print( type(exp),"unique values", np.unique(exp).shape , "\n\n")

100%|██████████| 40/40 [00:05<00:00,  7.59it/s]

<class 'numpy.ndarray'> unique values (1680,) 




In [15]:
exp = myFA4clf.explain(X_train_clf[:n_to_explain],labels=y_test_clf[:n_to_explain])
print( type(exp),"unique values", np.unique(exp).shape , "\n\n")


100%|██████████| 40/40 [00:38<00:00,  1.04it/s]

<class 'numpy.ndarray'> unique values (23911,) 




# regression  task

In [16]:
regressor = make_pipeline(MiniRocketMultivariate(n_jobs=1),
                    StandardScaler(),LinearRegression(n_jobs=-1))

regressor.fit(X_train_reg, y_train_reg)
print("metric is", regressor.score(X_test_reg,y_test_reg) )

metric is 0.5771726659034571


In [17]:
n_to_explain = 20
myLime4reg = LIME(regressor)
exp = myLime4reg.explain(X_train_reg[:n_to_explain] ,batch_size=4)
print( type(exp),"unique values", np.unique(exp).shape , "\n\n")

/home/davide/workspace/PhD/captum4aeon/tsCaptum/explainers.py:117: UserWarning:  batch_size set to 1 as suggested by Captum for Lime and KernelSHAP
  warnings.warn(" batch_size set to 1 as suggested by Captum for Lime and KernelSHAP")
100%|██████████| 20/20 [00:09<00:00,  2.17it/s]

<class 'numpy.ndarray'> unique values (722,) 




In [18]:
exp= myLime4reg.explain(X_train_reg[:n_to_explain] ,batch_size=4, n_segments=10)
print( type(exp),"unique values", np.unique(exp).shape , "\n\n",)

/home/davide/workspace/PhD/captum4aeon/tsCaptum/explainers.py:117: UserWarning:  batch_size set to 1 as suggested by Captum for Lime and KernelSHAP
  warnings.warn(" batch_size set to 1 as suggested by Captum for Lime and KernelSHAP")
 30%|███       | 6/20 [00:00<00:02,  6.33it/s]/home/davide/miniconda3/envs/TSCaptum/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.610e-02, tolerance: 1.407e-02
  model = cd_fast.enet_coordinate_descent(
100%|██████████| 20/20 [00:03<00:00,  6.14it/s]

<class 'numpy.ndarray'> unique values (501,) 




# Shapely Value Sampling
I thought let called it 'Shapely Value Sampling' in the library source code and import it as SHAP in the examples/tutorials for this code

In [19]:
regressor = make_pipeline(MiniRocketMultivariate(n_jobs=1),
                          StandardScaler(),LinearRegression(n_jobs=-1))

regressor.fit(X_train_reg, y_train_reg)
print("metric is", regressor.score(X_test_reg,y_test_reg) )

metric is 0.5788203574161792


In [ ]:
from tsCaptum.explainers import Shapley_Value_Sampling as SHAP
n_to_explain = 4
mySHAP4reg = SHAP(regressor)
exp = mySHAP4reg.explain(X_train_reg[:n_to_explain] ,batch_size=2)
print( type(exp),"unique values", np.unique(exp).shape , "\n\n")

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
exp = mySHAP4reg.explain(X_train_reg[:n_to_explain] ,batch_size=2, n_segments=5)
print( type(exp),"unique values", np.unique(exp).shape , "\n\n")